In [42]:
MAIN_DIR = "../output/bigdatacluster/experiments_adj/"

In [43]:
import os
import pandas as pd
import dill
import numpy as np

In [44]:
severity_folders = [x for x in os.listdir(MAIN_DIR) if os.path.isdir(os.path.join(MAIN_DIR, x))]

In [51]:
def analyze_rfe_features(path):
    dillfiles = [x for x in os.listdir(path) if x.endswith('.p')]
    csvfiles = [x for x in os.listdir(path) if x.endswith('.csv')]
    with open(os.path.join(path, 'FS_obj.p'), 'rb') as f:
        fs_obj = dill.load(f)
        
    df = pd.read_csv(os.path.join(path,'group_df_beforeFixation.csv'), index_col=0)
    
    df_score = pd.read_csv(os.path.join(path,'pseudo_metrics.csv'))
    df_score['avg'] = 0.5*(df_score['acc']+df_score['f1'])
    best_rfe = df_score.iloc[df_score['avg'].argmax()]['RFE']
    
    try:
        return best_rfe, dict(zip(df.columns[np.where(fs_obj[best_rfe].support_)[0]],fs_obj[best_rfe].estimator_.coef_[0]))
        
    except:
        return best_rfe, dict(zip(df.columns[np.where(fs_obj[best_rfe].support_)[0]],fs_obj[best_rfe].estimator_.feature_importances_))

            

In [55]:
severity_dict = {}

In [56]:
for sfolder in severity_folders:
    sev = sfolder.split('_')[0]
    fullpath = os.path.join(MAIN_DIR, sfolder,f"AgebetweenNonetNone_{sev}_TD_percentile_minmax")
    behavioral_folders = [x for x in os.listdir(fullpath) if os.path.isdir(os.path.join(fullpath, x))]
    behavioral_dict = {}
    for bfolder in behavioral_folders:
        full_bpath = os.path.join(fullpath, bfolder)
        best_rfe, rfe_dict = analyze_rfe_features(full_bpath)
        behavioral_dict[bfolder.split('_')[-1]] = (best_rfe, rfe_dict)
    severity_dict[sfolder] = behavioral_dict

# `severity_dict` is the base of the analysis 

# Question #1  